#developing a model to classify images of handwritten digits from the MNIST train set, and evaluating its performance on 500 random samples of MNIST test set:

In [43]:
install.packages("keras")

library(keras)

# Load the MNIST data
mnist <- dataset_mnist()
x_train <- mnist$train$x
y_train <- mnist$train$y
x_test <- mnist$test$x
y_test <- mnist$test$y

# Reshape input data to match the expected shape of the Keras model
x_train <- k_reshape(x_train, c(dim(x_train)[1], 28, 28, 1))
x_test <- k_reshape(x_test, c(dim(x_test)[1], 28, 28, 1))

# Define the deep learning model architecture
model <- keras_model_sequential() %>%
  layer_conv_2d(filters = 32, kernel_size = c(3, 3), activation = "relu", input_shape = c(28, 28, 1)) %>%
  layer_max_pooling_2d(pool_size = c(2, 2)) %>%
  layer_flatten() %>%
  layer_dense(units = 128, activation = "relu") %>%
  layer_dropout(rate = 0.5) %>%
  layer_dense(units = 10, activation = "softmax")

# Compile the model
model %>% compile(
  loss = "sparse_categorical_crossentropy",
  optimizer = optimizer_adam(),
  metrics = c("accuracy")
)

# Train the model on the MNIST training data
history <- model %>% fit(
  x_train, y_train,
  epochs = 10,
  batch_size = 128,
  validation_split = 0.2
)

# Use the model to predict class labels for 5000 images from a new dataset
new_data <- x_test[1:5000, , , ] # extract 5000 images from the test set
predictions <- predict(model, new_data) # make predictions on the new data

#Extract the predicted and actual classes for the first 500 images
predicted_classes <- apply(predictions[1:500, ], 1, which.max) - 1 # subtract 1 to convert to digits 0-9

actual_classes <- y_test[1:500]  # extract actual labels from the test set

# Store the predicted and actual classes in a dataframe
results_df <- data.frame(predicted = predicted_classes, actual = actual_classes)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



#performing a goodness-of-fit test using the chi-squared test statistic and a significance level of 0.05:

### observed counts for each class:

In [65]:
observed <- table(results_df$predicted)
observed


 0  1  2  3  4  5  6  7  8  9 
44 67 54 47 55 48 42 51 40 52 

### expected prpbability for each class:

In [66]:
expected <- rep(1/10, 10)
expected

[1] 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1

###chi-squared statistic, degrees of freedom, and p-value:

In [68]:
result <- chisq.test(observed, p=expected)
result


	Chi-squared test for given probabilities

data:  observed
X-squared = 10.96, df = 9, p-value = 0.2785


###critical value for df=9 and significance level=0.05:

In [76]:
critical_value <- qchisq((1-0.05), dim(observed)-1)
critical_value

[1] 16.91898